In [31]:
######################
## import libraries ##
######################
import numpy as np
np.random.seed(10815652)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [32]:
def simulate_dataset(n=100000, A_on_Y=0):
 
    ## A: observed binary intervention (0, 1)
    ## L: direct binary determinant of intervention A (0, 1)
    ## U1: binary unmeasured common cause of A and Y
    ## U2: binary unmeasured common cause of L and Y
    ## Y: countinuous outcome
    
    ## specify dataframe
    df = pd.DataFrame()
    
    ## specify variables Z, U1, and U2
    U1_split = 0.52
    U2_split = 0.23
    df['U1'] = np.random.choice([0, 1], size=n, replace=True, p=[U1_split, (1-U1_split)])
    df['U2'] = np.random.choice([0, 1], size=n, replace=True, p=[U2_split, (1-U2_split)])
    
    ## specify variable A
    lambda_0 = -2.32
    lambda_1 = 5.18
    df['A'] = 0
    df.loc[df['U1']==0, 'A'] = np.random.binomial(1, (1/(1+np.exp(-lambda_0))), size=df.loc[df['U1']==0, 'A'].shape[0])
    df.loc[df['U1']==1, 'A'] = np.random.binomial(1, (1/(1+np.exp(-(lambda_0+lambda_1)))), size=df.loc[df['U1']==1, 'A'].shape[0])

    ## specify variable L
    Beta_0 = -0.52
    Beta_1 = 2.32
    Beta_2 = 1.98
    df['L'] = 0
    df.loc[(df['U1']==0) & (df['U2']==0), 'L'] = np.random.binomial(1, (1/(1+np.exp(-Beta_0))), size=df.loc[(df['U1']==0) & (df['U2']==0), 'L'].shape[0])
    df.loc[(df['U1']==1) & (df['U2']==0), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_1)))), size=df.loc[(df['U1']==1) & (df['U2']==0), 'L'].shape[0])
    df.loc[(df['U1']==0) & (df['U2']==1), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_2)))), size=df.loc[(df['U1']==0) & (df['U2']==1), 'L'].shape[0])
    df.loc[(df['U1']==1) & (df['U2']==1), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_1+Beta_2)))), size=df.loc[(df['U1']==1) & (df['U2']==1), 'L'].shape[0])
    
    ## specify variable Y
    theta_0 = -0.5
    theta_1 = 5.78
    theta_2 = A_on_Y
    df['Y'] = theta_0 + (theta_1*df['U2']) + (theta_2*df['A']) + np.random.normal(0, 0.5, df.shape[0])
    
    df = df[['A', 'L', 'Y', 'U1', 'U2']]
    df_observed = df[['A', 'L', 'Y']].copy()
    
    return(df, df_observed)

In [33]:
#################################################################
## simulate dataset with no causal effect difference of A on Y ##
#################################################################
df, df_observed = simulate_dataset(n=1000000, A_on_Y=0)

In [34]:
#####################
## the "true" data ##
#####################
print(df.head(n=10))

   A  L         Y  U1  U2
0  0  0  5.247234   0   1
1  0  1  4.307400   0   1
2  0  1  4.864241   0   1
3  1  1  5.213704   1   1
4  1  1  4.334214   1   1
5  0  1  4.410975   0   1
6  1  1  5.623942   1   1
7  0  1 -0.863473   0   0
8  1  1 -0.259578   1   0
9  0  1  5.305411   1   1


In [35]:
##############################################
## data we get to "observe" in our analysis ##
##############################################
print(df_observed.head(n=10))

   A  L         Y
0  0  0  5.247234
1  0  1  4.307400
2  0  1  4.864241
3  1  1  5.213704
4  1  1  4.334214
5  0  1  4.410975
6  1  1  5.623942
7  0  1 -0.863473
8  1  1 -0.259578
9  0  1  5.305411


In [36]:
##############################################
## show L is associated with Intervention A ##
##############################################
model = smf.glm('A ~ L', data=df_observed, family=sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      A   No. Observations:              1000000
Model:                            GLM   Df Residuals:                   999998
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.5465e+05
Date:                Wed, 23 Dec 2020   Deviance:                   1.3093e+06
Time:                        12:45:20   Pearson chi2:                 1.00e+06
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3438      0.006   -227.188      0.0

In [37]:
######################################################################
## show L is associated with Outcome Y, condition on Intervention A ##
######################################################################
model = smf.ols(formula='Y ~ L + A', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                 5.852e+04
Date:                Wed, 23 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:45:21   Log-Likelihood:            -2.2717e+06
No. Observations:             1000000   AIC:                         4.543e+06
Df Residuals:                  999997   BIC:                         4.543e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3633      0.006    413.834      0.0

In [38]:
############################################
## Estimate the marginal effect of A on Y ##
############################################
model = smf.ols(formula='Y ~ A', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.271
Date:                Wed, 23 Dec 2020   Prob (F-statistic):              0.260
Time:                        12:45:22   Log-Likelihood:            -2.3270e+06
No. Observations:             1000000   AIC:                         4.654e+06
Df Residuals:                  999998   BIC:                         4.654e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.9527      0.004   1126.182      0.0

In [39]:
######################################################################
## simulate dataset with causal effect difference of A on Y of 0.12 ##
######################################################################
df, df_observed = simulate_dataset(n=1000000, A_on_Y=1.62)

In [40]:
##############################################
## show L is associated with Intervention A ##
##############################################
model = smf.glm('A ~ L', data=df_observed, family=sm.families.Binomial()).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      A   No. Observations:              1000000
Model:                            GLM   Df Residuals:                   999998
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.5484e+05
Date:                Wed, 23 Dec 2020   Deviance:                   1.3097e+06
Time:                        12:45:41   Pearson chi2:                 1.00e+06
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3396      0.006   -227.103      0.0

In [41]:
######################################################################
## show L is associated with Outcome Y, condition on Intervention A ##
######################################################################
model = smf.ols(formula='Y ~ L + A', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                 1.176e+05
Date:                Wed, 23 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:45:44   Log-Likelihood:            -2.2743e+06
No. Observations:             1000000   AIC:                         4.549e+06
Df Residuals:                  999997   BIC:                         4.549e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3607      0.006    412.839      0.0

In [42]:
############################################
## Estimate the marginal effect of A on Y ##
############################################
model = smf.ols(formula='Y ~ A', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                 1.067e+05
Date:                Wed, 23 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:45:49   Log-Likelihood:            -2.3293e+06
No. Observations:             1000000   AIC:                         4.659e+06
Df Residuals:                  999998   BIC:                         4.659e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.9453      0.004   1122.157      0.0